In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from fastai.dataset import *
from fastai.structured import *
from fastai.column_data import *

from math import ceil

from dateutil import parser

torch.cuda.set_device(0)

In [2]:
!export CUDA_LAUNCH_BLOCKING=1

In [3]:
PATH = "/home/paperspace/data/dog-breed-identification/"
sz=224
arch=resnext101_64
batch_size=5

## Exploratory Analysis and Feature Engineering

In [4]:
!ls {PATH}

item_categories.csv  models	      sample_submission.csv  test.csv
items.csv	     sales_train.csv  shops.csv		     tmp


In [5]:
sample_subm = pd.read_csv(f'{PATH}sample_submission.csv')
display(sample_subm.tail())

,ID,item_cnt_month
214195,214195,0.5
214196,214196,0.5
214197,214197,0.5
214198,214198,0.5
214199,214199,0.5


In [6]:
item_categories = pd.read_csv(f'{PATH}item_categories.csv')
items = pd.read_csv(f'{PATH}items.csv')
shops = pd.read_csv(f'{PATH}shops.csv')
train = pd.read_csv(f'{PATH}sales_train.csv')
test = pd.read_csv(f'{PATH}test.csv')

In [7]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [8]:
#### new try

In [9]:
#Make Monthly
train['date'] = pd.to_datetime(train['date'], format='%d.%m.%Y')
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year
train = train.drop(['date','item_price'], axis=1)
train = train.groupby([c for c in train.columns if c not in ['item_cnt_day']], as_index=False)[['item_cnt_day']].sum()
train = train.rename(columns={'item_cnt_day':'item_cnt_month'})
#Monthly Mean
shop_item_monthly_mean = train[['shop_id','item_id','item_cnt_month']].groupby(['shop_id','item_id'], as_index=False)[['item_cnt_month']].mean()
shop_item_monthly_mean = shop_item_monthly_mean.rename(columns={'item_cnt_month':'item_cnt_month_mean'})
#Add Mean Feature
train = pd.merge(train, shop_item_monthly_mean, how='left', on=['shop_id','item_id'])
#Last Month (Oct 2015)
shop_item_prev_month = train[train['date_block_num']==33][['shop_id','item_id','item_cnt_month']]
shop_item_prev_month = shop_item_prev_month.rename(columns={'item_cnt_month':'item_cnt_prev_month'})
shop_item_prev_month.head()
#Add Previous Month Feature
train = pd.merge(train, shop_item_prev_month, how='left', on=['shop_id','item_id']).fillna(0.)
#Items features
train = pd.merge(train, items, how='left', on='item_id')
#Item Category features
train = pd.merge(train, item_categories, how='left', on='item_category_id')
#Shops features
train = pd.merge(train, shops, how='left', on='shop_id')
train.head()

,date_block_num,shop_id,item_id,month,year,item_cnt_month,item_cnt_month_mean,item_cnt_prev_month,item_name,item_category_id,item_category_name,shop_name
0,0,0,32,1,2013,6.0,8.0,0.0,1+1,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
1,0,0,33,1,2013,3.0,3.0,0.0,1+1 (BD),37,Кино - Blu-Ray,"!Якутск Орджоникидзе, 56 фран"
2,0,0,35,1,2013,1.0,7.5,0.0,10 ЛЕТ СПУСТЯ,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
3,0,0,43,1,2013,1.0,1.0,0.0,100 МИЛЛИОНОВ ЕВРО,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
4,0,0,51,1,2013,2.0,2.5,0.0,100 лучших произведений классики (mp3-CD) (Dig...,57,Музыка - MP3,"!Якутск Орджоникидзе, 56 фран"


In [10]:
test = test.drop('ID', axis=1)

In [11]:
test['month'] = 11
test['year'] = 2015
test['date_block_num'] = 34
#Add Mean Feature
test = pd.merge(test, shop_item_monthly_mean, how='left', on=['shop_id','item_id']).fillna(0.)
#Add Previous Month Feature
test = pd.merge(test, shop_item_prev_month, how='left', on=['shop_id','item_id']).fillna(0.)
#Items features
test = pd.merge(test, items, how='left', on='item_id')
#Item Category features
test = pd.merge(test, item_categories, how='left', on='item_category_id')
#Shops features
test = pd.merge(test, shops, how='left', on='shop_id')
test['item_cnt_month'] = 0.
test.head()

,shop_id,item_id,month,year,date_block_num,item_cnt_month_mean,item_cnt_prev_month,item_name,item_category_id,item_category_name,shop_name,item_cnt_month
0,5,5037,11,2015,34,1.444444,0.0,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""",0.0
1,5,5320,11,2015,34,0.000000,0.0,ONE DIRECTION Made In The A.M.,55,Музыка - CD локального производства,"Вологда ТРЦ ""Мармелад""",0.0
2,5,5233,11,2015,34,2.000000,1.0,"Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""",0.0
3,5,5232,11,2015,34,1.000000,0.0,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры - XBOX 360,"Вологда ТРЦ ""Мармелад""",0.0
4,5,5268,11,2015,34,0.000000,0.0,"Need for Speed [PS4, русская версия]",20,Игры - PS4,"Вологда ТРЦ ""Мармелад""",0.0


In [12]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [13]:
items['item_category_id'] = items['item_category_id'].astype('category').cat.as_ordered()
items['item_name'] = items['item_name'].astype('category').cat.as_ordered()
items['item_id'] = items['item_id'].astype('category').cat.as_ordered()
item_categories['item_category_name'] = item_categories['item_category_name'].astype('category').cat.as_ordered()
shops['shop_id'] = shops['shop_id'].astype('category').cat.as_ordered()

# Training

## _Deep Learning Approach_

In [14]:
cat_vars = [
    'date_block_num',
    'month',              
    'year',               
    'shop_name',
]
contin_vars = [
    'item_cnt_month_mean',
    'item_cnt_prev_month',
    'item_cnt_month',
]
dep = 'item_cnt_month'

In [15]:
def rmse(y_pred, targ):
    return math.sqrt(((targ - y_pred)**2).mean())

In [16]:
date_block_num_cat = pd.api.types.CategoricalDtype(categories=[i for i in range(35)], ordered=True)

In [17]:
apply_cats(train, items)
apply_cats(test, items)
apply_cats(train, shops)
apply_cats(test, shops)
apply_cats(train, item_categories)
apply_cats(test, item_categories)

train['date_block_num'] = train['date_block_num'].astype(date_block_num_cat)
test['date_block_num'] = test['date_block_num'].astype(date_block_num_cat)

for v in cat_vars:
    train[v] = train[v].astype('category').cat.as_ordered()
for v in contin_vars:
    train[v] = train[v].astype('float32')
    test[v] = test[v].astype('float32')

apply_cats(test, train)
cat_vars += ['item_id', 'item_category_id', 'item_category_name', 'shop_id', 'item_name']

In [18]:
assert not train.isnull().any().any()

In [19]:
assert not test.isnull().any().any()

In [20]:
test.dtypes

shop_id                category
item_id                category
month                  category
year                   category
date_block_num         category
item_cnt_month_mean     float32
item_cnt_prev_month     float32
item_name              category
item_category_id       category
item_category_name     category
shop_name              category
item_cnt_month          float32
dtype: object

In [21]:
val = train[train['date_block_num']==33]

In [22]:
train = train[train['date_block_num']<33]

In [23]:
df, y, nas, mapper = proc_df(train, dep, do_scale=True)
val_df, val_y, nas, mapper = proc_df(
    val,
    dep,
    do_scale=True,
    mapper=mapper,
    na_dict=nas,
)
test_df, _, nas, mapper = proc_df(
    test,
    dep,
    do_scale=True,
    mapper=mapper,
    na_dict=nas
)

In [24]:
y_train = np.log1p(y.clip(0.,20.))
y_val = np.log1p(val_y.clip(0.,20.))

In [25]:
md = ColumnarModelData.from_data_frames(
    path=PATH,
    trn_df=df,
    trn_y=y_train,
    val_df=val_df,
    val_y=y_val,
    is_reg=True,
    is_multi=False,
    cat_flds=cat_vars,
    bs=256,
    test_df=test_df
)

In [26]:
cat_sz = [(c, len(train[c].cat.categories)+1) for c in cat_vars]

In [27]:
cat_sz

[('date_block_num', 36),
 ('month', 13),
 ('year', 4),
 ('shop_name', 61),
 ('item_id', 22171),
 ('item_category_id', 85),
 ('item_category_name', 85),
 ('shop_id', 61),
 ('item_name', 22171)]

In [28]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

In [29]:
emb_szs

[(36, 18),
 (13, 7),
 (4, 2),
 (61, 31),
 (22171, 50),
 (85, 43),
 (85, 43),
 (61, 31),
 (22171, 50)]

In [30]:
len(df.columns) - len(cat_vars)

2

In [31]:
m = md.get_learner(
    emb_szs=emb_szs,
    n_cont=len(df.columns) - len(cat_vars),
    emb_drop=.01,
    out_sz=1,
    szs=[1000, 500],
    drops=[.05, .01],
)

In [ ]:
m.lr_find(1e-6)

In [ ]:
m.sched.plot(1)

In [87]:
lr = 1e-6
m.fit(lrs=lr, n_cycle=3, metrics=[rmse], cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   rmse                           
    0      0.118113   0.120634   0.339356  
    1      0.112989   0.110912   0.32612                        
    2      0.109474   0.108682   0.322982                       
    3      0.107975   0.102524   0.313903                       
    4      0.108622   0.096987   0.305885                       
    5      0.11042    0.095142   0.303113                       
    6      0.1084     0.094828   0.302631                       



[array([0.09483]), 0.3026309589510541]

In [107]:
m.fit(lr, n_cycle=3, metrics=[rmse])

epoch      trn_loss   val_loss   rmse                           
    0      0.108013   0.090764   0.296348  
    1      0.106618   0.085964   0.288927                       
    2      0.10652    0.082406   0.283152                       



[array([0.08241]), 0.28315192553310353]

## Submit It!

In [108]:
sample_subm.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [109]:
reload_test = pd.read_csv(f'{PATH}test.csv')

In [110]:
apply_cats(reload_test, shops)
apply_cats(reload_test, items)

In [111]:
reload_test.dtypes

ID            int64
shop_id    category
item_id    category
dtype: object

In [112]:
copy_test = test.copy()

In [113]:
%%capture
ids_readded = pd.merge(reload_test, copy_test)

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

In [114]:
assert not ids_readded.isnull().any().any()

In [115]:
assert len(np.unique(ids_readded['ID'])) == len(ids_readded)

In [116]:
ids_readded.head()

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_month_mean,item_cnt_prev_month,item_name,item_category_id,item_category_name,shop_name,item_cnt_month
0,0,5,5037,11,2015,34,1.444444,0.0,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""",0.0
1,1,5,5320,11,2015,34,0.000000,0.0,ONE DIRECTION Made In The A.M.,55,Музыка - CD локального производства,"Вологда ТРЦ ""Мармелад""",0.0
2,2,5,5233,11,2015,34,2.000000,1.0,"Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""",0.0
3,3,5,5232,11,2015,34,1.000000,0.0,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры - XBOX 360,"Вологда ТРЦ ""Мармелад""",0.0
4,4,5,5268,11,2015,34,0.000000,0.0,"Need for Speed [PS4, русская версия]",20,Игры - PS4,"Вологда ТРЦ ""Мармелад""",0.0


In [117]:
ids_readded.head()

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_month_mean,item_cnt_prev_month,item_name,item_category_id,item_category_name,shop_name,item_cnt_month
0,0,5,5037,11,2015,34,1.444444,0.0,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""",0.0
1,1,5,5320,11,2015,34,0.000000,0.0,ONE DIRECTION Made In The A.M.,55,Музыка - CD локального производства,"Вологда ТРЦ ""Мармелад""",0.0
2,2,5,5233,11,2015,34,2.000000,1.0,"Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""",0.0
3,3,5,5232,11,2015,34,1.000000,0.0,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры - XBOX 360,"Вологда ТРЦ ""Мармелад""",0.0
4,4,5,5268,11,2015,34,0.000000,0.0,"Need for Speed [PS4, русская версия]",20,Игры - PS4,"Вологда ТРЦ ""Мармелад""",0.0


In [118]:
test_df.head()

,shop_id,item_id,month,year,date_block_num,item_cnt_month_mean,item_cnt_prev_month,item_name,item_category_id,item_category_name,shop_name
0,6,5038,11,NaN,NaN,-0.123836,-0.032005,NaN,20,NaN,NaN
1,6,5321,11,NaN,NaN,-0.345136,-0.032005,NaN,56,NaN,NaN
2,6,5234,11,NaN,NaN,-0.038721,0.058189,NaN,20,NaN,NaN
3,6,5233,11,NaN,NaN,-0.191929,-0.032005,NaN,24,NaN,NaN
4,6,5269,11,NaN,NaN,-0.345136,-0.032005,NaN,21,NaN,NaN


In [119]:
result = m.predict(is_test=True)
predictions = np.expm1(result)
display(predictions)

array([[1.36564],
       [0.70267],
       [1.75694],
       ...,
       [1.08913],
       [0.64791],
       [1.39985]], dtype=float32)

In [120]:
ids_readded['item_cnt_month'] = predictions

In [121]:
final_dataframe = ids_readded[['ID', 'item_cnt_month']]

In [122]:
final_dataframe.head()

,ID,item_cnt_month
0,0,1.365635
1,1,0.702674
2,2,1.756944
3,3,1.141582
4,4,0.752878


In [123]:
len(final_dataframe)

214200

In [124]:
len(np.unique(final_dataframe['ID']))

214200

In [125]:
SUBM = 'sub/'
os.makedirs(SUBM, exist_ok=True)
final_dataframe.to_csv(f'{SUBM}subm.gz', compression='gzip', index=False)

In [126]:
FileLink(f'{SUBM}subm.gz')

/home/paperspace/kaggle/competitive-data-science-predict-future-sales/sub/subm.gz